In [1]:
import numpy as np
np.set_printoptions(precision=4, suppress=True) # prettify numpy
import pandas as pd
pd.set_option('precision', 4) # prettify pandas
from binomial import binomial_tree, binomial_freq, terminal_coef

## 1. Write code to price a European Call option:

### a. The underlying stock that is currently trading at \\$95. The option has a strike price of \\$105 and 1 year maturity. Use the Binomial model with the parameters r=0 and 3 steps in the pricing process. 

Additionally, set the upper movement of the price (u) to an expression that will depend on the number on your group.

$$ u = (1.10 + \frac{Group Number}{100}), $$

Group 1, u = 1.11. Group 2 = 1.12. Group 3 = 1.13, …, Group 50 = 1.60

You can define $d = \frac{1}{u}$.

In [2]:
# Set strike price at $105
strike_price = 105 
# Calculate u, d, P_star
group_number = 2
u = 1.10 + group_number/100; d = 1/u
p = (1 - d) / (u - d)
p

0.4716981132075471

In [3]:
stock_prices_3steps = binomial_tree(
    N=3, # 3 steps
    S0=95, # underlying stock starts at $95,
    u=u,    
)['stock_prices']
df_stock_prices_3steps = pd.DataFrame(
    stock_prices_3steps
)
df_stock_prices_3steps.index.name = "Time Step"
df_stock_prices_3steps

,0,1,2,3
Time Step,,,,
0,95.0000,0.0000,0.000,0.0000
1,84.8214,106.4000,0.000,0.0000
2,75.7334,95.0000,119.168,0.0000
3,67.6191,84.8214,106.400,133.4682


In [4]:
terminal_coef_3steps = terminal_coef(
    N=3,
    u=u
)
terminal_coef_3steps

array([0.1475, 0.395 , 0.3526, 0.105 ])

In [5]:
option_payoff_3steps = np.where(
    stock_prices_3steps > strike_price, # if underlying stock is priced higher than strike
    stock_prices_3steps - strike_price, # then value = difference (buy low)
    0 # else 0 (leaving option unexercised)
)
df_option_payoff_3steps = pd.DataFrame(
    option_payoff_3steps
)
df_option_payoff_3steps.index.name = "Time Step"
df_option_payoff_3steps

,0,1,2,3
Time Step,,,,
0,0.0,0.0,0.000,0.0000
1,0.0,1.4,0.000,0.0000
2,0.0,0.0,14.168,0.0000
3,0.0,0.0,1.400,28.4682


In [6]:
# Value of the option at T0 = Sigma(probability for each path * value for each path)
sum(
    terminal_coef_3steps \
    * option_payoff_3steps[-1,:]
)

3.4814981494791057

### ANSWER: 
The value of the European call option, as described by the prompt, is therefore, $\approx 3.4815\$.

### b. Using the information from (a), show the value of the derivative, $H(\omega)$, for each of the paths.

In [7]:
df_option_payoff_3steps.iloc[3,:]

0     0.0000
1     0.0000
2     1.4000
3    28.4682
Name: 3, dtype: float64

### ANSWER:
The ending payoff $H(\omega)$ of the call option for each of the path is therefore:

- $0.00$ with 0 ups
- $0.00$ with 1 ups
- $1.40$ with 2 ups
- $\approx 28.47$ with 3 ups

## 2. Write code to price a European Put Option:

### a. Consider the same parameters as in section (a) in part (1) above but now with N=2. What is the price of the option?

In [8]:
stock_prices_2steps = binomial_tree(
    N=2, # 3 steps
    S0=95, # underlying stock starts at $95,
    u=u,    
)['stock_prices']
df_stock_prices_2steps = pd.DataFrame(
    stock_prices_2steps
)
df_stock_prices_2steps.index.name = "Time Step"
df_stock_prices_2steps

,0,1,2
Time Step,,,
0,95.0000,0.0,0.000
1,84.8214,106.4,0.000
2,75.7334,95.0,119.168


In [9]:
terminal_coef_2steps = terminal_coef(
    N=2,
    u=u,
)
terminal_coef_2steps

array([0.2791, 0.4984, 0.2225])

In [10]:
option_payoff_2steps = np.where(
    np.logical_and(
        stock_prices_2steps < strike_price, # if underlying stock is priced lower than strike
        stock_prices_2steps != 0 # and underlying stock is not 0.00 (technical limitation)
    ), 
    strike_price - stock_prices_2steps, # then value = difference (sell high)
    0 # else 0 (leaving option unexercised)
)
df_option_payoff_2steps = pd.DataFrame(
    option_payoff_2steps
)
df_option_payoff_2steps.index.name = "Time Step"
df_option_payoff_2steps

,0,1,2
Time Step,,,
0,10.0000,0.0,0.0
1,20.1786,0.0,0.0
2,29.2666,10.0,0.0


In [11]:
# Value of the option at T0 = Sigma(probability for each path * value for each path)
sum(
    terminal_coef_2steps \
    * option_payoff_2steps[-1,:]
)

13.152367390530443

In [12]:
option_payoff_2steps[2,:]

array([29.2666, 10.    ,  0.    ])

### ANSWER: 
The value of the European call option, as described by the prompt, is therefore, $\approx 12.3166\$.

The ending payoff $H(\omega)$ for each path is:

- $\approx 29.27$ with 0 ups (i.e. 2 downs)
- $10.00$ with 1 up (i.e. 1 downs)
- $0.00$ with 2 ups (i.e. 0 downs)

### b. Construct a Table (alike the ones you have in the notes) that includes, for each price path and each t when it corresponds: 

- the information on stock price evolution, $X_t(\omega)$, 
- the value of the option, $V_t^H(\omega)$, 
- the payoff of the option, $H(\omega)$, 
- and the hedging strategy, $\varphi_t^H$.

### ANSWER

The stock price evolution and corresponding option payoff can be seen below: <br>

| $$\omega$$ | $$X_0(\omega)$$ | $$X_1(\omega)$$ | $$X_2(\omega)$$ | $$H(\omega)$$ |
|-|-:|-:|-:|-:|
| $(u,u)$ | 95.0 | 106.4 | 119.168 | 0 |
| $(u,d)$ | 95.0 | 106.4 | 95.0 | 10.0 |
| $(d,u)$ | 95.0 | 84.8214 | 95.0 | 10.0 |
| $(d,d)$ | 95.0 | 84.8214 | 75.7334 | 29.2667 |

We also have $ P^* = \frac{1-d}{u-d} 
= \frac{1-\frac{1}{1.12}}{1.12 - \frac{1}{1.12}} 
\approx 0.4717 $ and $ 1 - P^* \approx 0.5283 $

Using that, we can calculate:

- $V_1^H(\{u,.\}) = V_1^H(\{u,u\})=V_1^H(\{u,d\}) \approx 0.4717 \cdot 0 + 0.5283 \cdot 10 \approx 5.283 $
- $V_1^H(\{d,.\}) = V_1^H(\{d,u\})=V_1^H(\{d,d\}) \approx 0.4717 \cdot 10 + 0.5283 \cdot 29.2666 \approx 20.1794 $
- $V_0^H \approx 0.4717^2 \cdot 0 + 2 \cdot 0.4717 \cdot 0.5283 \cdot 10 + 0.5283^2 \cdot 29.2666 \approx 13.1523 $

which is then tabulated as follows:

| $$\omega$$ | $$V_0^H$$ | $$V_1^H$$ | $$V_2^H$$ |
|-|-:|-:|-:|
| $(u;u)$ | 13.1523 | 5.283 | 0 |
| $(u;d)$ | 13.1523 | 5.283 | 10 |
| $(d;u)$ | 13.1523 | 20.1794 | 10 |
| $(d;d)$ | 13.1523 | 20.1794 | 29.2667 |

The hedging strategy is calculated by:

$$\varphi_t^H=\frac{(V_t^H-V_{t-1}^H)}{(X_t^H-X_{t-1}^H)}$$

We have the hedging strategy tabulated as follows:

| $$\omega$$ | $$\varphi_1^H$$ | $$\varphi_2^H $$ |
|-|-:|-:|
| $$(u;u)$$ | -0.6903 | -0.4138 |
| $$(u;d)$$ | -0.6903 | -0.4138 |
| $$(d;u)$$ | -0.6903 | -1.0000 |
| $$(d;d)$$ | -0.6903 | -1.0000 |